In [1]:
from copy import deepcopy
from functools import cache
from os import listdir
import json
import pickle
from datetime import datetime
from functools import partial

from scipy.special import softmax
import numpy as np
import math
from itertools import combinations

from typing import Any, List, Tuple
import time

import torch
import dataclasses


# |Set root for GPUDrive import
import os
import sys
from pathlib import Path

from traitlets import default

# Set working directory to the base directory 'gpudrive'
working_dir = Path.cwd()
while working_dir.name != 'gpudrive-CoDec':
    working_dir = working_dir.parent
    if working_dir == Path.home():
        raise FileNotFoundError("Base directory 'gpudrive' not found")
os.chdir(working_dir)
sys.path.append(str(working_dir))


# |GPUDrive imports
from gpudrive.utils.config import load_config
from examples.CoDec_Research.code.simulation.construal_main import generate_baseline_data, generate_selected_construal_traj, \
                                                                    get_constral_heurisrtic_values, generate_all_construal_trajnval
from examples.CoDec_Research.code.gpuDrive_utils import get_gpuDrive_vars
from examples.CoDec_Research.code.analysis.evaluate_construal_actions import evaluate_construals, get_best_construals_likelihood



RuntimeError: module compiled against ABI version 0x1000009 but this version of numpy is 0x2000000

RuntimeError: module compiled against ABI version 0x1000009 but this version of numpy is 0x2000000

2025-05-09 01:01:35.468093: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746766895.539091   75647 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746766895.561814   75647 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746766895.726634   75647 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746766895.726702   75647 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746766895.726703   75647 computation_placer.cc:177] computation placer alr

RuntimeError: module compiled against ABI version 0x1000009 but this version of numpy is 0x2000000

# Server Construal Values

In [ ]:
srFile = "examples/CoDec_Research/results/simulation_results/server/construal_vals_server.pickle"
with open(srFile, 'rb') as opn_file:
    default_values = pickle.load(opn_file)

In [4]:
default_values

{'dict_structure': '{scene_name: {construal_index: value}}',
 'tfrecord-00101-of-01000_71': {(0,): 1.0,
  (1,): 1.0,
  (5,): 1.0,
  (9,): 1.0,
  (10,): 1.0,
  (42,): 1.0,
  (43,): 1.0,
  (0, 1, 5, 9, 10, 42, 43): 1.0},
 'tfrecord-00117-of-01000_240': {(0,): 0.9555555555555556,
  (1,): 1.0,
  (2,): 1.0,
  (3,): 0.9,
  (4,): 0.9777777777777777,
  (5,): 0.9,
  (7,): 0.9111111111111111,
  (8,): 0.9444444444444444,
  (9,): 0.9555555555555556,
  (10,): 0.9777777777777777,
  (11,): 0.9222222222222223,
  (12,): 0.9777777777777777,
  (13,): 0.9333333333333333,
  (14,): 1.0,
  (15,): 1.0,
  (16,): 0.9777777777777777,
  (17,): 0.9333333333333333,
  (0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17): 1.0},
 'tfrecord-00143-of-01000_179': {(0,): 0.9777777777777777,
  (1,): 1.0,
  (2,): 0.9777777777777777,
  (3,): 0.9777777777777777,
  (4,): 1.0,
  (5,): 0.9666666666666667,
  (6,): 0.9777777777777777,
  (7,): 1.0,
  (8,): 0.9777777777777777,
  (9,): 1.0,
  (10,): 1.0,
  (11,): 1.0,
  (12,):

# Server Construal Trajs

In [2]:
srFile = "examples/CoDec_Research/results/simulation_results/server/all_constr_obs_server.pickle"
with open(srFile, 'rb') as opn_file:
    traj_obs = pickle.load(opn_file)

In [ ]:
for scene_name in traj_obs.keys():
    if scene_name == 'dict_structure':
        continue
    

In [ ]:
# import os
# from pathlib import Path

# # Set working directory to the base directory 'gpudrive'
# working_dir = Path.cwd()
# while working_dir.name != 'gpudrive-CoDec':
#     working_dir = working_dir.parent
#     if working_dir == Path.home():
#         raise FileNotFoundError("Base directory 'gpudrive' not found")
# os.chdir(working_dir)

In [ ]:
from os import listdir
import json
from genericpath import isfile

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# Visualization Logic (Dynamic)

In [ ]:
def animate_scenario(curr_data: dict, filename: str):
    fig, ax = plt.subplots(1, 1, dpi=300)
    ax.set_xticks([])
    ax.set_yticks([])

    # |Plot roads
    min_x, min_y = (np.inf, np.inf)
    for road_obj in curr_data['roads']:
        road_xy = [(rdict['x'], rdict['y']) for rdict in road_obj['geometry']]
        color = {
            'crosswalk': 'grey',
            'driveway': 'grey',
            'lane': 'darkgrey',
            'road_edge': 'black',
            'road_line': 'yellow',
            'speed_bump': 'grey',
            'stop_sign': 'red'
        }[road_obj['type']]
        rx, ry = zip(*road_xy)
        if min(rx) < min_x: min_x = min(rx)
        if min(ry) < min_y: min_y = min(ry)
        ax.plot(rx, ry, color=color, lw=.2)
    ax.set_title(filename.split('/')[-1])

    # |Get vehicle info
    #2# |Loop through the rest
    for obj_num, veh_obj in enumerate(curr_data['objects']):
        veh_pos = [[posdict['x'],posdict['y']] if posdict['x']!=-10000 else [min_x,min_y] for posdict in veh_obj['position']]
        vx, vy = zip(*veh_pos)
        if obj_num == 0:
            all_vx = np.array([vx])
            all_vy = np.array([vy])
        else:
            all_vx = np.append(all_vx, [vx], axis=0)
            all_vy = np.append(all_vy, [vy], axis=0)

    # Set all vehicles except controlled vehicle to blue
    veh_colors = ['blue']*all_vx.shape[0]
    veh_colors[curr_data['metadata']['sdc_track_index']] = 'red'

    scat = ax.scatter(all_vx[:,0], all_vy[:,0], marker='o', s=veh_obj['width']*.1, lw=1, alpha=1, c=veh_colors)

    # |Plot veh traj
    def animate_func(timestep):        
        curr_offset = np.stack([all_vx[:,timestep],all_vy[:,timestep]]).transpose()
        scat.set_offsets(curr_offset)
    anim = animation.FuncAnimation(fig, animate_func, repeat=True, frames=all_vx.shape[1] - 1, interval=50)

    anim.save(filename=filename, fps=15)
    plt.close()


In [ ]:
dirpath = "data/processed/construal/"
outputDir = "data/processed/Viz_construed_sim/"

for scene_num, (scene_name, scene_info) in enumerate(traj_obs.items()):
    if scene_name == 'dict_structure':
        continue
    print(f"\rprocesseing file {scene_num}", end="", flush=True)
    with open(dirpath+scene_name+'.json', 'r') as opn_file:
        data = json.load(opn_file)
    for construal_indcs, construal_info in scene_info.items():
        for sample_num, sample_traj in construal_info.items():
            plot_title = scene_name+'_'+str(construal_indcs)+'_'+str(sample_num)
            sample_traj = sample_traj.cpu().numpy()
            # animate_scenario(data, outputDir+plot_title+'.gif', sample_traj, construal_indcs, (0), plot_title)
    

processeing file 1tfrecord-00101-of-01000_71_(0,)_0
tfrecord-00101-of-01000_71_(0,)_1
tfrecord-00101-of-01000_71_(0,)_2
tfrecord-00101-of-01000_71_(0,)_3
tfrecord-00101-of-01000_71_(0,)_4
tfrecord-00101-of-01000_71_(0,)_5
tfrecord-00101-of-01000_71_(0,)_6
tfrecord-00101-of-01000_71_(0,)_7
tfrecord-00101-of-01000_71_(0,)_8
tfrecord-00101-of-01000_71_(0,)_9
tfrecord-00101-of-01000_71_(0,)_10
tfrecord-00101-of-01000_71_(0,)_11
tfrecord-00101-of-01000_71_(0,)_12
tfrecord-00101-of-01000_71_(0,)_13
tfrecord-00101-of-01000_71_(0,)_14
tfrecord-00101-of-01000_71_(0,)_15
tfrecord-00101-of-01000_71_(0,)_16
tfrecord-00101-of-01000_71_(0,)_17
tfrecord-00101-of-01000_71_(0,)_18
tfrecord-00101-of-01000_71_(0,)_19
tfrecord-00101-of-01000_71_(0,)_20
tfrecord-00101-of-01000_71_(0,)_21
tfrecord-00101-of-01000_71_(0,)_22
tfrecord-00101-of-01000_71_(0,)_23
tfrecord-00101-of-01000_71_(0,)_24
tfrecord-00101-of-01000_71_(0,)_25
tfrecord-00101-of-01000_71_(0,)_26
tfrecord-00101-of-01000_71_(0,)_27
tfrecord-001

# Visualization Logic (Static)

In [ ]:

def plot_scenario(curr_data: dict, 
                  filename: str,
                  sample_traj: np.array = None,
                  construal_indcs: tuple = None,
                  ego_indcs: tuple =None,
                  plot_title: str = None, 
                  ):
    fig, ax = plt.subplots(1, 1, dpi=300)
    ax.set_xticks([])
    ax.set_yticks([])

    # |Plot Roads
    # for road_obj in curr_data['roads']:
    #     road_xy = [(rdict['x'], rdict['y']) for rdict in road_obj['geometry']]
    #     color = {
    #         'crosswalk': 'grey',
    #         'driveway': 'grey',
    #         'lane': 'darkgrey',
    #         'road_edge': 'black',
    #         'road_line': 'yellow',
    #         'speed_bump': 'grey',
    #         'stop_sign': 'red'
    #     }[road_obj['type']]
    #     rx, ry = zip(*road_xy)
    #     # print(min(rx), max(rx))
    #     ax.plot(rx, ry, color=color, lw=.2)
    # ax.set_title(plot_title)

    # |Plot Vehicles
    if sample_traj is None:
        for veh_num, veh_obj in enumerate(curr_data['objects']):
            veh_pos = [[posdict['x'],posdict['y']] for posdict in veh_obj['position'] if posdict['x']!=-10000]
            vx, vy = zip(*veh_pos)
            if veh_num != curr_data['metadata']['sdc_track_index']:
                ax.plot(vx, vy, marker='.', ms=veh_obj['width']*.1, lw=1, alpha=0.3)
            else:
                ax.plot(vx, vy, marker='.', ms=veh_obj['width']*.1, lw=1, alpha=1, color='grey')
    else:
        veh_width = curr_data['objects'][0]['width']*.1
        for veh_num, veh_obj in enumerate(sample_traj):
            veh_obj = veh_obj.tolist()
            veh_pos = [curr_pos_ for curr_pos_ in veh_obj if curr_pos_[0]!=-11000]
            vx, vy = zip(*veh_pos)
            print(max(vy), min(vy))
            if veh_num in construal_indcs:
                ax.plot(vx, vy, marker='.', ms=veh_width, lw=1, alpha=0.3)
            elif veh_num in ego_indcs:
                ax.plot(vx, vy, marker='.', ms=veh_width, lw=1, alpha=1, color='red')
            
    # plt.show()
    fig.savefig(filename)
    plt.close()


In [51]:
dirpath = "data/processed/construal/"
outputDir = "examples/CoDec_Research/results/simulation_results/Viz_construed_sim/"

for scene_num, (scene_name, scene_info) in enumerate(traj_obs.items()):
    if scene_name == 'dict_structure':
        continue
    print(f"\rprocesseing file {scene_num}", end="", flush=True)
    with open(dirpath+scene_name+'.json', 'r') as opn_file:
        data = json.load(opn_file)
    for construal_indcs, construal_info in scene_info.items():
        for sample_num, sample_traj in construal_info.items():
            plot_title = scene_name+'_'+str(construal_indcs)+'_'+str(sample_num)
            sample_traj = sample_traj.cpu().numpy()
            plot_scenario(data, outputDir+plot_title+'.png', sample_traj, construal_indcs, (0,), plot_title)
            break


processeing file 1-8.078649520874023 -9.494912147521973
-8.86328125 -9.2578125
6.1953125 6.1953125
-4.87109375 -14.4453125
-14.73828125 -14.73828125
-9.359375 -10.16796875
-18.3515625 -18.3515625
13.11328125 13.11328125
-13.61328125 -13.61328125
-4.2265625 -4.78125
-9.53125 -9.9765625
9.83984375 9.83984375
-21.31640625 -21.31640625
-0.1328125 -0.1328125
-22.44921875 -22.44921875
-11.3203125 -11.3203125
10.38671875 10.38671875
-10.3203125 -10.3203125
16.08984375 16.08984375
-27.94140625 -27.94140625
-17.015625 -17.015625
-10.67578125 -10.67578125
-30.01953125 -30.01953125
-32.12109375 -32.12109375
-10.4296875 -10.4296875
-37.00390625 -37.00390625
0.59765625 0.59765625
18.77734375 18.77734375
-9.94921875 -9.94921875
-24.59375 -24.59375
-9.3671875 -9.3671875
2.20703125 2.20703125
-40.0390625 -40.0390625
-8.09375 -8.09375
-34.70703125 -34.70703125
-42.85546875 -42.85546875
-36.21484375 -36.21484375
-8.234375 -28.53515625
-45.2890625 -45.2890625
19.27734375 19.27734375
-28.0859375 -28.08593

KeyboardInterrupt: 